In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
from pathlib import Path
import os

current_path = Path().resolve().parent
print(current_path)

In [2]:
def find_pt_file(folder_path):
    # List all files in the folder
    all_files = os.listdir(folder_path)
    
    # Filter out the .pt files
    pt_filename = [f for f in all_files if f.endswith('.pt')][0]

    return int(pt_filename.split(".")[0].split('_')[-1])

def get_best_model_f1(folder_path):

    best_f1 = 0
    best_iter = 0
    
    for i in range(10):
        df = pd.read_csv(folder_path / f'test_results_exp_{i}.csv')
        true_labels = df['true_label']
        predictions = df['prediction']

        score = f1_score(true_labels, predictions, average='macro')
        if score > best_f1:
            best_f1 = score
            best_iter = i

    return best_iter, best_f1


def get_labels_and_predictions(corpus, model, task):

    results_path = current_path / 'results' / f"{corpus}-{model}-{task}/"
    
    best_iter, best_f1 = get_best_model_f1(results_path) # based on f1-score on testing set

    print(corpus, model, task, round(best_f1, 3))

    df = pd.read_csv(results_path / f'test_results_exp_{best_iter}.csv')
    true_labels = df['true_label']
    predictions = df['prediction']

    return true_labels, predictions

In [ ]:
labels1, predictions1 = get_labels_and_predictions('ibm', 'bert', 'multi')
labels2, predictions2 = get_labels_and_predictions('ukp', 'bert', 'multi')

# Function to plot a confusion matrix
def plot_confusion_matrix(ax, labels_true, labels_pred, title, prec_or_rec):
    cm = confusion_matrix(labels_true, labels_pred)

    names_list = ['Correct', 'Flipped', 'Neutralized', 'Polarized']

    cm_to_plot = cm
    formato = "d"
    if prec_or_rec == 'recall':
        cm_to_plot = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100 # cm_row_norm
        formato = ".0f"
    elif prec_or_rec == 'precision':
        cm_to_plot = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :] * 100 # cm_col_norm
        formato = ".0f"

    sns.heatmap(cm_to_plot, annot=True, fmt=formato, cmap="Blues", ax=ax, cbar = False, annot_kws={"size": 14})
    ax.set_title(title)
    ax.set_xlabel('Predicho')
    ax.set_xticklabels(names_list)
    ax.set_yticklabels(names_list)
    ax.set_ylabel('Correcto')

# Create subplots for three confusion matrices in a row
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot each confusion matrix
metrc = 'recmfefeall'
plot_confusion_matrix(ax1, labels1, predictions1, r'$EVI^{\dagger}$', metrc)
plot_confusion_matrix(ax2, labels2, predictions2, r'$UKP^{\dagger}$', metrc)

# # Adjust layout and display the plot
plt.tight_layout()

# plt.savefig(str((current_path) / 'figures' / 'ch8-confusionMatrices.pdf'), format='pdf')

plt.show()
